# Ridge Regression

1. Latar Belakang
   
Pemetaan dan penyediaan data geografis yang akurat memainkan peran penting dalam banyak aspek, terutama dalam konteks administrasi, perencanaan wilayah, dan layanan berbasis lokasi. Dalam era digital ini, kebutuhan akan data geospasial yang lengkap dan akurat semakin meningkat, terutama bagi lembaga pemerintah, sektor bisnis, dan organisasi non-profit yang menggunakan data tersebut untuk analisis spasial, perencanaan strategis, dan peningkatan layanan publik.

Namun, proses pengumpulan data geografis sering kali menghadapi tantangan dalam hal ketersediaan dan akurasi. Data yang tidak lengkap atau memiliki informasi koordinat geografis yang kosong dapat menghambat analisis dan interpretasi data. Untuk itu, metode geocoding dan prediksi menggunakan machine learning dapat menjadi solusi yang efektif dalam melengkapi data koordinat yang hilang, sehingga mempercepat proses pengolahan dan meningkatkan keakuratan data spasial.

2. Rumusan Masalah
Adapun permasalahan yang dihadapi dalam proyek ini adalah:

Bagaimana cara mendapatkan koordinat latitude dan longitude dari data alamat yang tersedia dengan cepat dan akurat?
Bagaimana metode machine learning dapat digunakan untuk memprediksi koordinat latitude dan longitude pada data yang tidak lengkap?
Bagaimana mengukur keakuratan prediksi koordinat yang dihasilkan dibandingkan dengan data geografis yang valid?

3. Tujuan
Tujuan dari proyek ini adalah sebagai berikut:

Mengimplementasikan metode geocoding untuk memperoleh koordinat latitude dan longitude berdasarkan alamat yang tersedia.
Mengembangkan model machine learning yang dapat digunakan untuk memprediksi nilai koordinat geografis (latitude dan longitude) pada data yang hilang.
Mengevaluasi keakuratan model prediksi dengan membandingkannya terhadap data koordinat geografis yang sudah valid, sehingga dapat memberikan insight mengenai keandalan model machine learning dalam konteks pengisian data geografis yang tidak lengkap.

In [ ]:
import pandas as pd
from geopy.geocoders import ArcGIS
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_absolute_error

nom = ArcGIS(timeout=10)

df = pd.read_excel('1000 Longlat.xlsx')

def buat_alamat(row):
    return f"{row['provinsi']}, {row['kabupaten']}, {row['kecamatan']}, {row['kode_pos']}, Indonesia"

df['Alamat Lengkap'] = df.apply(buat_alamat, axis=1)

df['latitude'] = None
df['longitude'] = None

def geocode_address(index, row):
    try:
        lokasi = nom.geocode(row['Alamat Lengkap'])
        if lokasi:
            return index, lokasi.latitude, lokasi.longitude
        else:
            return index, None, None
    except Exception as e:
        print(f"Error pada baris {index}: {e}")
        return index, None, None

start_time = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(geocode_address, index, row) for index, row in df.head(200).iterrows()]
    
    for future in as_completed(futures):
        index, lat, lon = future.result()
        df.at[index, 'latitude'] = lat
        df.at[index, 'longitude'] = lon

end_time = time.time()

print(f"Waktu yang diperlukan untuk proses Geocoding: {end_time - start_time} detik")

df.to_excel('1000_Longlat_Geocoded_Sample.xlsx', index=False)

df_geocoded = df.dropna(subset=['latitude', 'longitude'])

X = df_geocoded[['provinsi', 'kabupaten', 'kecamatan', 'kode_pos']]
y_latitude = df_geocoded['latitude']
y_longitude = df_geocoded['longitude']


X_train_val_lat, X_test_lat, y_train_val_lat, y_test_lat = train_test_split(X, y_latitude, test_size=0.2, random_state=42)
X_train_lat, X_val_lat, y_train_lat, y_val_lat = train_test_split(X_train_val_lat, y_train_val_lat, test_size=0.25, random_state=42)

X_train_val_lon, X_test_lon, y_train_val_lon, y_test_lon = train_test_split(X, y_longitude, test_size=0.2, random_state=42)
X_train_lon, X_val_lon, y_train_lon, y_val_lon = train_test_split(X_train_val_lon, y_train_val_lon, test_size=0.25, random_state=42)

preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), ['provinsi', 'kabupaten', 'kecamatan', 'kode_pos'])
])

svd = TruncatedSVD(n_components=15)

model_latitude = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('svd', svd),
                                 ('regressor', Ridge())])

model_longitude = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('svd', svd),
                                  ('regressor', Ridge())])

param_grid = {
    'regressor__alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
    'regressor__fit_intercept': [True, False]
}

grid_search_lat = GridSearchCV(model_latitude, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search_lat.fit(X_train_lat, y_train_lat)

grid_search_lon = GridSearchCV(model_longitude, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search_lon.fit(X_train_lon, y_train_lon)

model_latitude_best = grid_search_lat.best_estimator_
model_longitude_best = grid_search_lon.best_estimator_

y_lat_pred = model_latitude_best.predict(X_test_lat)
y_lon_pred = model_longitude_best.predict(X_test_lon)

latitude_mae = mean_absolute_error(y_test_lat, y_lat_pred)
longitude_mae = mean_absolute_error(y_test_lon, y_lon_pred)

print(f"Latitude MAE (Data Test) dengan Ridge: {latitude_mae}")
print(f"Longitude MAE (Data Test) dengan Ridge: {longitude_mae}")

df_predict = df[df['latitude'].isnull()].copy()

X_predict = df_predict[['provinsi', 'kabupaten', 'kecamatan', 'kode_pos']]

predicted_latitudes = model_latitude_best.predict(X_predict)
predicted_longitudes = model_longitude_best.predict(X_predict)

df.loc[df['latitude'].isnull(), 'latitude'] = predicted_latitudes
df.loc[df['longitude'].isnull(), 'longitude'] = predicted_longitudes

df.to_excel('1000_Longlat_Predicted_Ridge_SVD_Improved.xlsx', index=False)

valid_data = pd.read_excel('Longlat valid.xlsx')

merged_data = pd.merge(df, valid_data, on='npsn', suffixes=('_predicted', '_valid'))

latitude_mae_valid = mean_absolute_error(merged_data['latitude_valid'], merged_data['latitude_predicted'])
longitude_mae_valid = mean_absolute_error(merged_data['longitude_valid'], merged_data['longitude_predicted'])

latitude_range = merged_data['latitude_valid'].max() - merged_data['latitude_valid'].min()
longitude_range = merged_data['longitude_valid'].max() - merged_data['longitude_valid'].min()

latitude_accuracy = 100 - (latitude_mae_valid / latitude_range * 100)
longitude_accuracy = 100 - (longitude_mae_valid / longitude_range * 100)

print(f"Latitude MAE (Data Valid) dengan Ridge: {latitude_mae_valid}")
print(f"Longitude MAE (Data Valid) dengan Ridge: {longitude_mae_valid}")
print(f"Latitude Accuracy: {latitude_accuracy}%")
print(f"Longitude Accuracy: {longitude_accuracy}%")
